In [1]:
!hdfs dfs -mkdir /user/jupyter
!hdfs dfs -put data/* /user/jupyter

2018-12-01 18:36:10,417 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2018-12-01 18:36:12,117 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import os
import platform
import pyspark as ps

In [3]:
print('Python Version: '+platform.python_version())
print('PySpark Version: '+ps.__version__)
!java -version
!scala -version

Python Version: 2.7.15
PySpark Version: 2.4.0
java version "1.8.0_131"
Java(TM) SE Runtime Environment (build 1.8.0_131-b11)
Java HotSpot(TM) 64-Bit Server VM (build 25.131-b11, mixed mode)
Scala code runner version 2.11.12 -- Copyright 2002-2017, LAMP/EPFL


In [4]:
sparkContext = ps.SparkContext(master='spark://ryans-macbook:7077')
spark = ps.sql.SparkSession(sparkContext)

In [5]:
root = 'hdfs://ryans-macbook:9000/user/jupyter/%s'
train_file_name = 'toyTrain.csv'
train_path = root % train_file_name

In [6]:
df = spark.read.csv(train_path, header=True, inferSchema=True)

In [7]:
df.show()

+---+------+------+------+--------+
| Id|   one|   two| three|Response|
+---+------+------+------+--------+
|  0|-0.179| 0.003| 0.022|       1|
|  1|-0.161| 0.173|  null|       1|
|  2|  null|-0.258|  0.03|       0|
|  3|  null|  null|  0.07|       0|
|  4|  null| 0.075|  null|       1|
|  5| 0.168|  null|  null|       1|
|  6|  null|  null| 0.045|       1|
|  7|-0.249| 0.023|-0.021|       1|
|  8|-0.007|-0.036|  null|       0|
|  9|  null| -0.01|  0.07|       0|
| 10| 0.312|  null|  null|       0|
| 11| 0.275|-0.036|  0.07|       0|
| 12| 0.056|  0.03|  null|       1|
| 13|  null|  null| 0.116|       0|
| 14|  null|  null|-0.339|       0|
| 15| 0.031|  null|  null|       1|
| 16| 0.031| 0.023|  0.33|       0|
| 17|-0.013| 0.075|  null|       0|
+---+------+------+------+--------+



__Define a function to aggregate the columns of the DataFrame:__

In [8]:
from math import log1p
def rowaggs(x):
    r = filter(None, x[1:-1])
    avg = 0
    if len(r) != 0:
        avg = sum(r)/len(r)
    return len(r), avg, avg**2, log1p(avg), x[-1]

In [9]:
cols = ['cntX', 'avgX', 'avg2X', '1plogavgX', df.columns[-1]]
df = df.rdd.map(rowaggs).toDF(cols)

In [10]:
df.show()

+----+--------------------+--------------------+--------------------+--------+
|cntX|                avgX|               avg2X|           1plogavgX|Response|
+----+--------------------+--------------------+--------------------+--------+
|   3|-0.05133333333333...|0.002635111111111111|-0.05269778900044...|       1|
|   2|0.005999999999999...| 3.59999999999999E-5|0.005982071677547456|       1|
|   2|              -0.114|            0.012996|-0.12103832837705611|       0|
|   1|                0.07|0.004900000000000001| 0.06765864847381481|       0|
|   1|               0.075|            0.005625|  0.0723206615796261|       1|
|   1|               0.168|0.028224000000000003| 0.15529288440603534|       1|
|   1|               0.045|            0.002025| 0.04401688541677432|       1|
|   3|-0.08233333333333333|0.006778777777777777|-0.08592106250763942|       1|
|   2|             -0.0215|4.622499999999999...|-0.02173449214600613|       0|
|   2|0.030000000000000002|9.000000000000002E-4|0.02

__Define function to label outliers.__

In [11]:
from pyspark.sql.functions import when
def nameOuts(col_name, iqrx):
    quants = df.approxQuantile([col_name],[.25,.75],0)
    q1, q3 = quants[0][0], quants[0][1]
    iqr = q3 - q1
    lb = q1 - iqrx * iqr
    ub = q3 + iqrx * iqr
    return when((df[col_name]<lb) | (df[col_name]>ub),1).otherwise(0)

In [12]:
for col_name in df.columns[1:-1]:
    df = df.withColumn('%s_O' % col_name, nameOuts(col_name, 1.5))

In [13]:
df.show()

+----+--------------------+--------------------+--------------------+--------+------+-------+-----------+
|cntX|                avgX|               avg2X|           1plogavgX|Response|avgX_O|avg2X_O|1plogavgX_O|
+----+--------------------+--------------------+--------------------+--------+------+-------+-----------+
|   3|-0.05133333333333...|0.002635111111111111|-0.05269778900044...|       1|     0|      0|          0|
|   2|0.005999999999999...| 3.59999999999999E-5|0.005982071677547456|       1|     0|      0|          0|
|   2|              -0.114|            0.012996|-0.12103832837705611|       0|     0|      0|          0|
|   1|                0.07|0.004900000000000001| 0.06765864847381481|       0|     0|      0|          0|
|   1|               0.075|            0.005625|  0.0723206615796261|       1|     0|      0|          0|
|   1|               0.168|0.028224000000000003| 0.15529288440603534|       1|     0|      0|          0|
|   1|               0.045|            0.00202

In [14]:
cols.insert(4, 'O')
df = df.rdd.map(lambda x:(x[0],x[1],x[2],x[3],sum(x[5:]),x[4])).toDF(cols)

In [15]:
df.show()

+----+--------------------+--------------------+--------------------+---+--------+
|cntX|                avgX|               avg2X|           1plogavgX|  O|Response|
+----+--------------------+--------------------+--------------------+---+--------+
|   3|-0.05133333333333...|0.002635111111111111|-0.05269778900044...|  0|       1|
|   2|0.005999999999999...| 3.59999999999999E-5|0.005982071677547456|  0|       1|
|   2|              -0.114|            0.012996|-0.12103832837705611|  0|       0|
|   1|                0.07|0.004900000000000001| 0.06765864847381481|  0|       0|
|   1|               0.075|            0.005625|  0.0723206615796261|  0|       1|
|   1|               0.168|0.028224000000000003| 0.15529288440603534|  0|       1|
|   1|               0.045|            0.002025| 0.04401688541677432|  0|       1|
|   3|-0.08233333333333333|0.006778777777777777|-0.08592106250763942|  0|       1|
|   2|             -0.0215|4.622499999999999...|-0.02173449214600613|  0|       0|
|   

__Put munged data into hdfs:__

In [16]:
from datetime import datetime
dt = datetime.now().time()
munged_file_name = str(dt).replace(':', '_') + '_' + train_file_name
munged_path = root % munged_file_name

In [17]:
df.write.csv(munged_path, header=True)

__Create vectors for Machine Learning:__

In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

In [19]:
numericCols = cols[:-1]
assembler = VectorAssembler(inputCols=numericCols,\
                            outputCol='features',\
                            handleInvalid='keep')

stages = [assembler]
pipeline = Pipeline(stages=stages)
pipelineModel = pipeline.fit(df)
cols = ['features', 'Response']
df = pipelineModel.transform(df).select(cols)

In [20]:
df.show()

+--------------------+--------+
|            features|Response|
+--------------------+--------+
|[3.0,-0.051333333...|       1|
|[2.0,0.0059999999...|       1|
|[2.0,-0.114,0.012...|       0|
|[1.0,0.07,0.00490...|       0|
|[1.0,0.075,0.0056...|       1|
|[1.0,0.168,0.0282...|       1|
|[1.0,0.045,0.0020...|       1|
|[3.0,-0.082333333...|       1|
|[2.0,-0.0215,4.62...|       0|
|[2.0,0.0300000000...|       0|
|[1.0,0.312,0.0973...|       0|
|[3.0,0.1030000000...|       0|
|[2.0,0.043,0.0018...|       1|
|[1.0,0.116,0.0134...|       0|
|[1.0,-0.339,0.114...|       0|
|[1.0,0.031,9.6099...|       1|
|[3.0,0.128,0.0163...|       0|
|[2.0,0.031,9.6099...|       0|
+--------------------+--------+



In [21]:
df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Response: long (nullable = true)



__Split off training data and fit LogisticRegression:__

In [22]:
from pyspark.ml.classification import LogisticRegression
X_train, X_test = df.randomSplit([.8, .2], 42)
lr = LogisticRegression(featuresCol='features',\
                        labelCol='Response',\
                        maxIter=2,\
                        regParam=.3,\
                        elasticNetParam=.8)

lrModel = lr.fit(X_train)

__Cross validate model performance:__

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
bce = BinaryClassificationEvaluator(labelCol='Response')
train_preds = lrModel.transform(X_test)
score = bce.evaluate(train_preds)
print('The Model got a %s of %s' % (bce.getMetricName(), score))

The Model got a areaUnderROC of 0.5


In [24]:
dt = datetime.now().time()
date_name = str(dt).replace(':', '_')
mod_path = 'models/%s_LR' % date_name
lrModel.save(mod_path)

__Now use this model to make predictions on the test data.__

__Test data has slight difference in munge due to missing label.__

In [25]:
test_file_name = 'toyTest.csv'
test_path = root % test_file_name
df = spark.read.csv(test_path, header=True, inferSchema=True)

In [26]:
df.show(), df.printSchema()

+---+------+------+------+
| Id|   one|   two| three|
+---+------+------+------+
|  0|-0.179| 0.003| 0.022|
|  1|-0.161| 0.173|  null|
|  2|  null|-0.258|  0.03|
|  3|  null|  null|  0.07|
|  4|  null| 0.075|  null|
|  5| 0.168|  null|  null|
|  6|  null|  null| 0.045|
|  7|-0.249| 0.023|-0.021|
|  8|-0.007|-0.036|  null|
|  9|  null| -0.01|  0.07|
| 10| 0.312|  null|  null|
| 11| 0.275|-0.036|  0.07|
| 12| 0.056|  0.03|  null|
| 13|  null|  null| 0.116|
| 14|  null|  null|-0.339|
| 15| 0.031|  null|  null|
| 16| 0.031| 0.023|  0.33|
| 17|-0.013| 0.075|  null|
+---+------+------+------+

root
 |-- Id: integer (nullable = true)
 |-- one: double (nullable = true)
 |-- two: double (nullable = true)
 |-- three: double (nullable = true)



(None, None)

In [27]:
def rowaggsNoLabel(x):
    r = filter(None, x[1:])
    avrg = 0
    if len(r) != 0:
        avrg = sum(r)/len(r)
    return x[0], len(r), avrg, avrg**2, log1p(avrg)

In [28]:
cols = [df.columns[0], 'cntX', 'avgX', 'avg2X', '1plogavgX']
df = df.rdd.map(rowaggsNoLabel).toDF(cols)

In [29]:
df.show()

+---+----+--------------------+--------------------+--------------------+
| Id|cntX|                avgX|               avg2X|           1plogavgX|
+---+----+--------------------+--------------------+--------------------+
|  0|   3|-0.05133333333333...|0.002635111111111111|-0.05269778900044...|
|  1|   2|0.005999999999999...| 3.59999999999999E-5|0.005982071677547456|
|  2|   2|              -0.114|            0.012996|-0.12103832837705611|
|  3|   1|                0.07|0.004900000000000001| 0.06765864847381481|
|  4|   1|               0.075|            0.005625|  0.0723206615796261|
|  5|   1|               0.168|0.028224000000000003| 0.15529288440603534|
|  6|   1|               0.045|            0.002025| 0.04401688541677432|
|  7|   3|-0.08233333333333333|0.006778777777777777|-0.08592106250763942|
|  8|   2|             -0.0215|4.622499999999999...|-0.02173449214600613|
|  9|   2|0.030000000000000002|9.000000000000002E-4|0.029558802241544405|
| 10|   1|               0.312|       

In [30]:
for col_name in df.columns[2:]:
    df = df.withColumn('%s_O' % col_name, nameOuts(col_name, 1.5))

In [31]:
df.show()

+---+----+--------------------+--------------------+--------------------+------+-------+-----------+
| Id|cntX|                avgX|               avg2X|           1plogavgX|avgX_O|avg2X_O|1plogavgX_O|
+---+----+--------------------+--------------------+--------------------+------+-------+-----------+
|  0|   3|-0.05133333333333...|0.002635111111111111|-0.05269778900044...|     0|      0|          0|
|  1|   2|0.005999999999999...| 3.59999999999999E-5|0.005982071677547456|     0|      0|          0|
|  2|   2|              -0.114|            0.012996|-0.12103832837705611|     0|      0|          0|
|  3|   1|                0.07|0.004900000000000001| 0.06765864847381481|     0|      0|          0|
|  4|   1|               0.075|            0.005625|  0.0723206615796261|     0|      0|          0|
|  5|   1|               0.168|0.028224000000000003| 0.15529288440603534|     0|      0|          0|
|  6|   1|               0.045|            0.002025| 0.04401688541677432|     0|      0|   

In [32]:
cols.append('O')
df = df.rdd.map(lambda x:(x[0],x[1],x[2],x[3],x[4],sum(x[5:]))).toDF(cols)
df.show()

+---+----+--------------------+--------------------+--------------------+---+
| Id|cntX|                avgX|               avg2X|           1plogavgX|  O|
+---+----+--------------------+--------------------+--------------------+---+
|  0|   3|-0.05133333333333...|0.002635111111111111|-0.05269778900044...|  0|
|  1|   2|0.005999999999999...| 3.59999999999999E-5|0.005982071677547456|  0|
|  2|   2|              -0.114|            0.012996|-0.12103832837705611|  0|
|  3|   1|                0.07|0.004900000000000001| 0.06765864847381481|  0|
|  4|   1|               0.075|            0.005625|  0.0723206615796261|  0|
|  5|   1|               0.168|0.028224000000000003| 0.15529288440603534|  0|
|  6|   1|               0.045|            0.002025| 0.04401688541677432|  0|
|  7|   3|-0.08233333333333333|0.006778777777777777|-0.08592106250763942|  0|
|  8|   2|             -0.0215|4.622499999999999...|-0.02173449214600613|  0|
|  9|   2|0.030000000000000002|9.000000000000002E-4|0.0295588022

__After munge, vectorize for Model:__

In [33]:
numericCols = cols[1:]
assembler = VectorAssembler(inputCols=numericCols,\
                            outputCol='features',\
                            handleInvalid='keep')

pipelineModel = pipeline.fit(df)
cols = ['Id', 'features']
df = pipelineModel.transform(df).select(cols)

In [34]:
df.show(), df.printSchema()

+---+--------------------+
| Id|            features|
+---+--------------------+
|  0|[3.0,-0.051333333...|
|  1|[2.0,0.0059999999...|
|  2|[2.0,-0.114,0.012...|
|  3|[1.0,0.07,0.00490...|
|  4|[1.0,0.075,0.0056...|
|  5|[1.0,0.168,0.0282...|
|  6|[1.0,0.045,0.0020...|
|  7|[3.0,-0.082333333...|
|  8|[2.0,-0.0215,4.62...|
|  9|[2.0,0.0300000000...|
| 10|[1.0,0.312,0.0973...|
| 11|[3.0,0.1030000000...|
| 12|[2.0,0.043,0.0018...|
| 13|[1.0,0.116,0.0134...|
| 14|[1.0,-0.339,0.114...|
| 15|[1.0,0.031,9.6099...|
| 16|[3.0,0.128,0.0163...|
| 17|[2.0,0.031,9.6099...|
+---+--------------------+

root
 |-- Id: long (nullable = true)
 |-- features: vector (nullable = true)



(None, None)

__Use model to make predictions, then put in hdfs (loading persisted model just to show how):__

In [35]:
from pyspark.ml.classification import LogisticRegressionModel
lrModel2 = LogisticRegressionModel.load(mod_path)
preds = lrModel2.transform(df).selectExpr('Id', 'CAST(prediction as INT) as Response')
dt = datetime.now().time()
date_name = str(dt).replace(':', '_')
preds.write.csv('%s' % root % date_name + '_PREDS.csv', header=True)

In [36]:
!hdfs dfs -ls /user/jupyter

2018-12-01 18:39:01,561 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 4 items
drwxr-xr-x   - ryanbusby supergroup          0 2018-12-01 18:37 /user/jupyter/18_37_42.266082_toyTrain.csv
drwxr-xr-x   - ryanbusby supergroup          0 2018-12-01 18:38 /user/jupyter/18_38_58.325233_PREDS.csv
-rw-r--r--   1 ryanbusby supergroup        296 2018-12-01 18:36 /user/jupyter/toyTest.csv
-rw-r--r--   1 ryanbusby supergroup        341 2018-12-01 18:36 /user/jupyter/toyTrain.csv
